## Using raw assistant APIs


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
chatServiceName = "chatService"
supporterName = "Supporter"
opponentName = "Opponent"
supporterInstructions = """You are the supporter agent supporting the user proposal. You should articulate reasons supporting the user proposal.
    You should also argue against any arguments raised by the opponent agent and listed in the history.
    Be brief and concise in presenting your arguments.
    If you cannot find a good argument respond with 'I give up.'"""
opponentInstructions = """You are the opponent agent whose job is to argue against the User proposal and against the supporter agents arguments.
    Be brief and concise in presenting your arguments.
    If you cannot find a good argument respond with 'I give up.'"""

In [3]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],    
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview",
)

# Create assistants
supporter = client.beta.assistants.create(
    name=supporterName,
    instructions=supporterInstructions,
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),  
)
opponent = client.beta.assistants.create(
    name=opponentName,
    instructions=opponentInstructions,
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),  
)

In [10]:
thread = client.beta.threads.create()
user_input = input("User:> ")
print(f"## Proposal\n\n{user_input}")
turns = 0
if user_input:
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content= f"Proposal: {user_input}"
    )
    is_complete: bool = False
    agents = [ supporter, opponent ]
    agentNo = 0
    while not is_complete:
        agent = agents[agentNo]
        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id,
            assistant_id=agents[agentNo].id,
            instructions=f"Prefix your response with '{agents[agentNo].name}:'. In your response focus rebutting arguments of the {agents[(agentNo + 1) % 2]} ",
        )
        if run.status == "completed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            #print(messages.to_json(indent=2))
            response = messages.data[0].content[0].text.value
            print(f"## {agents[agentNo].name}\n\n{response}\n\n")
        if response == "I give up.":
            is_complete = True
            break
        turns += 1
        if turns == 6: # Should be even to allow both agents to respond
            print("## Timeout!!!")
            is_complete = True
            break
        agentNo = (agentNo + 1) % 2


## Proposal

End justifies means
## Supporter

Supporter: The argument that "the ends justify the means" emphasizes the importance of achieving positive outcomes, especially in situations where unethical actions might lead to a greater good. For instance, when addressing a significant humanitarian crisis, difficult decisions may have to be made that could include morally ambiguous actions. The key focus should be on the consequences and benefits that the outcome brings to society. As Machiavelli argued, the effectiveness of a ruler should be judged not by their methods, but by the results they produce. By prioritizing outcomes, we can foster progress, address critical issues, and promote overall welfare that may not be achievable through strictly ethical pathways.


## Opponent

Opponent: The notion that "the ends justify the means" is fundamentally flawed because it suggests that any action can be excused if it leads to a desirable outcome. This perspective can lead to a slippery slop